# Naive Bayes - emails classification

**Here we use sklearn.naive_bayes to train a spam classifier. The data use is open source data.**

First, we import the required packages.

In [21]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

We construct a function to read the files. The folder containig the data, has two folders; one contains the spam emails and the other the ham (or no spam) emails. Each email is in a separate file.

In [22]:
def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)
            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message

This function creates our data frame.

In [23]:
def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('emails/ham', 'ham'))

Our dataframe looks likes this:

In [24]:
data.head()

,message,class
emails/spam/00249.5f45607c1bffe89f60ba1ec9f878039a,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",spam
emails/spam/00373.ebe8670ac56b04125c25100a36ab0510,ATTENTION: This is a MUST for ALL Computer Use...,spam
emails/spam/00214.1367039e50dc6b7adb0f2aa8aba83216,This is a multi-part message in MIME format.\n...,spam
emails/spam/00210.050ffd105bd4e006771ee63cabc59978,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,spam
emails/spam/00033.9babb58d9298daa2963d4f514193d7d6,This is the bottom line. If you can GIVE AWAY...,spam


We split our data into training and test data.

In [39]:
import sklearn.model_selection as model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(data['message'].values, data['class'].values, train_size=0.65,test_size=0.35, random_state=101)

We will use a CountVectorizer tokenize the collection of messages in the emails in order to create a vocabulary. It generates vectors representing the number of ocurrences of each word in each email. 

In [40]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(X_train)

Now we use a MultinomialNB classifier. We call fit() in order to train the model.

In [41]:
classifier = MultinomialNB()
targets = y_train
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let us test it our model.

In [43]:
counts = vectorizer.transform(X_test)
predictions = classifier.predict(counts)

And compute its performance.

In [44]:
from sklearn import metrics
performance = metrics.accuracy_score(y_test, predictions)
print(performance)

0.9561904761904761
